### What is the goal

Predicting Oscars winner in top 5 categories (Best Picture, Director, Screenplay, Actor and Actress). Assuming these are easy to predict.

March 12th deadline. The end goal is to bet on them though? Managing risk with bets


### Background information

+ Everybody votes on best pictures (6000) total
+ Actors nominate actors, film editors nominate film editors, etc...

Full rules https://www.oscars.org/sites/oscars/files/95th_oscars_complete_rules.pdf

Basic rules:
+ Run-off elimination: Each vote is a ranking of the nominations. The contender with the lowest number of first place votes get removed, and second place votes get counted in their place (does this creates "bland" movies? how to quantify? https://www.youtube.com/watch?v=AfIxihGOaQ8&ab_channel=Vox ) -> new in 2009


### Potential data sources
+ Need an Oscar dataset
Movie | Year of release | Nominated for x ... | Winning of x... | The name at each category | Win/nominated at other awards | ethnicity of each | Snubbed before (?) -> Previously nominated | Awarded before

+ Ratings? 


+ The theme (sypnosis)

+ IMDB ratings/reviews 

+ opensubtitles.org: Really deep dive

+ How to get past betting odds

### Human intuition

+ How to incorporate guilt for snubbing actors?
+ Movies that win are period pieces, usually about the movie industry, watched by white males
-> Can we categorize the main themes: slavery, holocaust, mentally challenged, war, disease, tortured artist, sexual abuse, maid, 

+ Cannes prestige?
+ Golden Globes are given by press or critics? while Oscars is an invitation only club

Extra?
+ Animations: They don't even watch this -> Can I get it based on box office numbers? They probably watch it with their kids. 2018 opened up for all voters instead of just animators


### Related projects/datasets
https://www.kaggle.com/datasets/unanimad/the-oscar-award/code?datasetId=384513&sortBy=dateRun&tab=profile

https://github.com/MateVaradi/OscarPrediction & https://www.kaggle.com/datasets/matevaradi/oscar-prediction-dataset

https://www.kaggle.com/code/baynebrannen/oscar-winner-prediction-randomforestregressor/notebook

https://www.kaggle.com/competitions/oscarprediction/data?select=oscardata_bestpicture_test.csv

https://journal-doi.org/10.731/pcbi.1007742/

https://aisel.aisnet.org/cgi/viewcontent.cgi?article=1185&context=ecis2008

file:///C:/Users/tanng/Downloads/4427-949-4385-1-10-20181130.pdf

https://arxiv.org/pdf/2001.00878.pdf

In [77]:
import imdb_scraper as imdb
import json
import pandas as pd

#custom
import random
import time


event_cache = {} # key (event_id, year)
award_cache = {} # key (entity_type, entity_id)

person_cache = {} 
title_cache = {}

In [44]:
list(event_cache.values())[0]

IndexError: list index out of range

In [49]:
print(json.dumps(imdb.get_event("ev0000003", "2006", event_cache), indent=4))

Cant't normalize this category name Best Writing, Original Screenplay
Cant't normalize this category name Best Achievement in Art Direction
Cant't normalize this category name Best Achievement in Music Written for Motion Pictures, Original Song
Cant't normalize this category name Best Achievement in Sound Mixing
Cant't normalize this category name Best Achievement in Sound Editing
Cant't normalize this category name Best Short Film, Animated
Cant't normalize this category name Best Short Film, Live Action
Cant't normalize this category name Best Documentary, Short Subjects
Cant't normalize this category name Best Documentary, Features
Cant't normalize this category name Best Foreign Language Film of the Year
Cant't normalize this category name Best Animated Feature Film of the Year
Cant't normalize this category name Best Achievement in Makeup
[
    {
        "award": "Oscar",
        "category": "PICTURE",
        "noms": [
            {
                "name": "Crash",
              

In [98]:
### Build up all best picture nominees
picture_nominees = []
for year in range(1960, 2023):
    try: 
        categories = imdb.get_event("ev0000003", str(year), event_cache)
        for cat in categories:
            if cat["award"] == "Oscar" and imdb.normalize_oscar_category(cat["category"]) ==  "PICTURE":
                for nom in cat["noms"]:
                    picture_nominees.append({
                        "nominated_date": pd.Timestamp(
                            year=year,
                            month=3,
                            day=10
                        ),
                        "name": nom["name"],
                        "id": nom["entity_id"], 
                        "is_winner": nom["is_winner"],
                        "producer_count": len(nom["secondary_names"]),
                        "producer_names": nom["secondary_names"],
                        "producer_ids": nom["secondary_ids"],
                        **get_title_info(nom["entity_id"], title_cache)
                    })
                break
        print(f"Running on year {year}")
    except Exception as error:
        print(f"Error on {year} {error=}, type {type(error)}")
        break
    time.sleep(random.randrange(1))


picture_df = pd.DataFrame(picture_nominees)
picture_df

Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Running on year 1960
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Can't normalize {name}
Error on 1961 error=TypeError("'NoneType' object is not subscriptable"), type <class 'TypeError'>


,nominated_date,name,id,is_winner,producer_count,producer_names,producer_ids,certificate,release_date,runtime,...,directors_count,directors_names,directors_ids,writers_count,writers_names,writers_ids,actors_names,actors_ids,box_office,aspect_ratio
0,1960-03-10,Ben-Hur,tt0052618,True,1,[Sam Zimbalist],[nm0956547],G,1959-11-18,12720,...,1,[William Wyler],[nm0943758],6,"[Lew Wallace, Karl Tunberg, Gore Vidal]","[nm0908753, nm0876562, nm0000683]",[],[],74437720,2.75 : 1
1,1960-03-10,Anatomy of a Murder,tt0052561,False,1,[Otto Preminger],[nm0695937],R,1959-07-13,9660,...,1,[Otto Preminger],[nm0695937],2,"[Wendell Mayes, John D. Voelker]","[nm0562606, nm0871261]",[],[],194,1.85 : 1
2,1960-03-10,Room at the Top,tt0053226,False,2,"[John Woolf, James Woolf]","[nm0941153, nm0941150]",14+,1959-01-22,7020,...,1,[Jack Clayton],[nm0002338],3,"[Neil Paterson, John Braine, Mordecai Richler]","[nm0665513, nm0104050, nm0725006]",[],[],0,1.66 : 1
3,1960-03-10,The Diary of Anne Frank,tt0052738,False,1,[George Stevens],[nm0828419],PG,1959-04-17,10800,...,1,[George Stevens],[nm0828419],4,"[Frances Goodrich, Albert Hackett, Anne Frank]","[nm0329304, nm0352443, nm0290833]",[],[],0,2.35 : 1
4,1960-03-10,The Nun's Story,tt0053131,False,1,[Henry Blanke],[nm0087533],PG,1959-07-18,8940,...,1,[Fred Zinnemann],[nm0003593],2,"[Robert Anderson, Kathryn Hulme]","[nm0027355, nm0401520]",[],[],0,1.78 : 1
5,1961-03-10,The Apartment,tt0053604,True,1,[Billy Wilder],[nm0000697],PG,1960-06-29,7500,...,1,[Billy Wilder],[nm0000697],2,"[Billy Wilder, I.A.L. Diamond]","[nm0000697, nm0224634]",[],[],18778738,2.35 : 1


In [ ]:
def acta

In [1]:
picture_df.describe()

NameError: name 'picture_df' is not defined

In [97]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Referer": "https://targetwebsite.com/page1",
}

BASE_IMDB_URL = "https://www.imdb.com"

import requests
from bs4 import BeautifulSoup
from json import JSONDecoder
import builtins


### Get basic info from title
def get_title_info(title_id, title_cache=None):
    if title_cache is not None and title_id in title_cache:
        return title_cache[title_id]
    
    page = requests.get(f"{BASE_IMDB_URL}/title/{title_id}/", headers=HEADERS)
    soup = BeautifulSoup(page.text, "html.parser")
    
    script = soup.find("script", {"id": "__NEXT_DATA__", "type": "application/json"}).text
    script_json = JSONDecoder().decode(script)["props"]["pageProps"]
    
    with open("script2.json", "w", encoding="utf-8") as f:
        f.write(json.dumps(script_json, indent=4))
    # Summary above
    summary_data = script_json["aboveTheFoldData"]
    summary = {
        "certificate": summary_data["certificate"]["rating"],
        "release_date": pd.Timestamp(
            day=summary_data["releaseDate"]["day"],
            month=summary_data["releaseDate"]["month"],
            year=summary_data["releaseDate"]["year"],
        ),
        "runtime": summary_data["runtime"]["seconds"],
        "rating_ave": summary_data["ratingsSummary"]["aggregateRating"],
        "rating_count": summary_data["ratingsSummary"]["voteCount"],
        "metacritic": summary_data["metacritic"]["metascore"]["score"],
        "genres": [genre["id"] for genre in summary_data["genres"]["genres"]],
        "directors_count": 0,
        "directors_names": [],
        "directors_ids": [],
        "writers_count": 0,
        "writers_names": [],
        "writers_ids": [],
    }
    for credit in summary_data["principalCredits"]:
        if credit["category"]["id"] == "director":
            summary["directors_count"] = credit["totalCredits"]
            summary["directors_names"] = [director["name"]["nameText"]["text"] for director in credit["credits"]]
            summary["directors_ids"] = [director["name"]["id"] for director in credit["credits"]]
        elif credit["category"]["id"] == "writer":
            summary["writers_count"] = credit["totalCredits"]
            summary["writers_names"] = [writer["name"]["nameText"]["text"] for writer in credit["credits"]]
            summary["writers_ids"] = [writer["name"]["id"] for writer in credit["credits"]]
    
    # Main below
    main_data = script_json["mainColumnData"]
    main = {
        "actors_names": [],
        "actors_ids": [],
        "box_office": safe_dget(main_data, "worldwideGross", "total", "amount"),
        "aspect_ratio": safe_dget(safe_lget(safe_dget(main_data, "technicalSpecifications", "aspectRatios", "items"), 0), "aspectRatio"),   
    }
    
    return {
        **summary,
        **main,
    }

def safe_dget(dct, *keys):
    for key in keys:
        try:
            dct = dct[key]
        except Exception as error:
            return 0
    return dct


def safe_lget(lis, index):
    try: 
        return lis[index]
    except Exception as error:
        return {}

get_title_info("tt0053226")
### Get awards from title and parse to DF

{'certificate': '14+',
 'release_date': Timestamp('1959-01-22 00:00:00'),
 'runtime': 7020,
 'rating_ave': 7.6,
 'rating_count': 6619,
 'metacritic': 84,
 'genres': ['Drama', 'Romance'],
 'directors_count': 1,
 'directors_names': ['Jack Clayton'],
 'directors_ids': ['nm0002338'],
 'writers_count': 3,
 'writers_names': ['Neil Paterson', 'John Braine', 'Mordecai Richler'],
 'writers_ids': ['nm0665513', 'nm0104050', 'nm0725006'],
 'actors_names': [],
 'actors_ids': [],
 'box_office': 0,
 'aspect_ratio': '1.66 : 1'}

In [93]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.imdb.com/title/tt0120737/'

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

title = soup.find('h1').text.strip()
year = soup.find('span', class_='TitleYear').text.strip('()')
rating = soup.find('span', itemprop='ratingValue').text.strip()

print('Title:', title)
print('Year:', year)
print('Rating:', rating)




AttributeError: 'NoneType' object has no attribute 'text'

In [24]:
movie_awards = imdb.get_awards(imdb.EntityType.TITLE, 'tt0167260', award_cache)
print(json.dumps(movie_awards, indent=4))

[
    {
        "result": "Winner",
        "award": "Oscar",
        "category": "Best Picture",
        "nominee": [
            "Barrie M. Osborne",
            "Peter Jackson",
            "Fran Walsh"
        ]
    },
    {
        "result": "Winner",
        "award": "Oscar",
        "category": "Best Director",
        "nominee": [
            "Peter Jackson"
        ]
    },
    {
        "result": "Winner",
        "award": "Oscar",
        "category": "Best Writing, Adapted Screenplay",
        "nominee": [
            "Fran Walsh",
            "Philippa Boyens",
            "Peter Jackson"
        ]
    },
    {
        "result": "Winner",
        "award": "Oscar",
        "category": "Best Film Editing",
        "nominee": [
            "Jamie Selkirk"
        ]
    },
    {
        "result": "Winner",
        "award": "Oscar",
        "category": "Best Art Direction-Set Decoration",
        "nominee": [
            "Grant Major",
            "(art director)",
            "

In [91]:
b = {}
c = [1]
b[0] = 'a'
b[0]
c[0]

1

In [88]:
b

{0: 'a'}

In [ ]:
"""

Movie table (predicting best Picture)

1. Main info: name, id, release date (-> time to Oscars),  runtime
2. Producers info: names, ids, count, average nominations/wins across all the main awards (fill when scraping actors)
3. Actors info: names, ids, count, historical nominations/wins across all the main awards
4. Nomination, wins (yes/no) accross all the categories and awards
5. Director info: names, ids, count, average nominations/wins, total nomintations, wins
6. Scores: count/average across IMDB, RottenTomatoes, LetterBoxD
7. [Stretch] Sentimental analysis from IMDB? RottenTomatoes? LetterBox
8. Box office

"""



"""
Individuals table (predicting best Actor/Actress/)

1. Previous nominations
2. 
"""